In [ ]:
from graphviz import Digraph

In [ ]:
import math
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self._prev = set(_children)
    self._backward = lambda: None
    self.grad = 0
    self._op = _op
    self.label = label
  
  def __add__(self, other):  # self + other
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += out.grad
      other.grad += out.grad
    out._backward = _backward
    
    return out
  
  def __mul__(self, other):  # self * other
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out._backward = _backward
    return out
  
  def __pow__(self, other): # self ** other
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data ** other.data, (self, other), '**')

    def _backward():
      self.grad += other.data * self.data**(other.data - 1) * out.grad
      # other.grad = math.log(self.data) * math.exp(other.data) * out.grad

    out._backward = _backward
    return out

  def relu(self): # relu(self)
    out = Value(self.data if self.data > 0 else 0, (self,), 'ReLU')

    def _backward():
      self.grad += (out.data > 0) * out.grad

    out._backward = _backward
    return out

  def tanh(self):
    x = self.data
    t = (math.exp(2*x)-1) / (math.exp(2*x)+1)
    out = Value(t, (self,), 'tanh')

    def _backward():
      self.grad += (1 - (out.data ** 2)) * out.grad

    out._backward = _backward
    return out

  def backward(self):
    visited = set()

    topo = []
    def build_topo(v):
        if v not in visited:
            visited.add(v)
            for child in v._prev:
                build_topo(child)
            topo.append(v)
    build_topo(self)
    topo = reversed(topo)

    self.grad = 1.0
    for v in topo:
      v._backward()

  def __neg__(self): # -self
    return self * -1

  def __radd__(self, other): # other + self
    return self + other

  def __sub__(self, other): # self - other
    return self + (-other)

  def __rsub__(self, other): # other - self
    return self + (-other)

  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __rtruediv__(self, other): # other / self
    return  self * other**-1

  def __repr__(self):
    return f"Value(data={self.data}, grad={self.grad})"
  


In [ ]:
import random

class Module:
    def zero_grad(self):
      for p in self.parameters():
        p.grad = 0

    def parameters(self):
      pass

class Neuron(Module):
    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin= nonlin

    def __call__(self, x):
        out = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return out.tanh() if self.nonlin else out

    def parameters(self):
      return self.w + [self.b]

    def __repr__(self):
        return ''

class Layer(Module):

    def __init__(self, nin, nout, nonlin=True):
      self.neurons = [Neuron(nin, nonlin) for _ in range(nout)]

    def __call__(self, x):
      outs = [neuron(x) for neuron in self.neurons]
      return outs[0] if len(outs) == 1 else outs

    def parameters(self):
      return sum([n.parameters() for n in self.neurons], [])

    def __repr__(self):
      return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):
    def __init__(self, nin, nouts):
      sz = [nin] + nouts
      self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def __call__(self, x):
      for layer in self.layers:
        x = layer(x)
      return x

    def parameters(self):
      return [p for layer in self.layers for p in layer.parameters()]
    
    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]


# len(nn.parameters()) == (3+1)*4 + (4+1) * 4 + (4+1)*1

In [ ]:
nn = MLP(3, [4,4,4,1])
for itr in range(300):
  nn.zero_grad()

  ypred = [nn(x) for x in xs]
  loss = sum([(ygt-yout)**2 for ygt, yout in zip(ys, ypred)])

  loss.backward()
  print(loss.data)
  for p in nn.parameters():
    p.data -= 0.02 * p.grad


In [ ]:
def trace(root):
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph('svg', graph_attr={'rankdir': 'LR'})

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    dot.node(name= uid, label="{%s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      dot.node(name=uid + n._op, label=n._op)
      dot.edge(uid + n._op, uid)
  
  for n1, n2 in edges:
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot 